In [100]:
import pandas as pd

In [101]:
data = pd.read_csv('SeoulBikeData.csv', encoding='ISO-8859-1')

## 1. Описательный анализ

In [102]:
data.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


### Пропуски

In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

### Распределение показателей

In [104]:
data.describe()

,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,704.602055,11.500000,12.882922,58.226256,1.724909,1436.825799,4.073813,0.569111,0.148687,0.075068
std,644.997468,6.922582,11.944825,20.362413,1.036300,608.298712,13.060369,0.868746,1.128193,0.436746
min,0.000000,0.000000,-17.800000,0.000000,0.000000,27.000000,-30.600000,0.000000,0.000000,0.000000
25%,191.000000,5.750000,3.500000,42.000000,0.900000,940.000000,-4.700000,0.000000,0.000000,0.000000
50%,504.500000,11.500000,13.700000,57.000000,1.500000,1698.000000,5.100000,0.010000,0.000000,0.000000
75%,1065.250000,17.250000,22.500000,74.000000,2.300000,2000.000000,14.800000,0.930000,0.000000,0.000000
max,3556.000000,23.000000,39.400000,98.000000,7.400000,2000.000000,27.200000,3.520000,35.000000,8.800000


### Выбросы

In [105]:
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data['Date'] = data['Date'].astype(int) / 10**9 

In [106]:
from scipy.stats import zscore

In [107]:
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
outliers = (data[numerical_cols].apply(zscore).abs() > 3).sum()
outliers

Date                           0
Rented Bike Count             67
Hour                           0
Temperature(°C)                0
Humidity(%)                    0
Wind speed (m/s)              63
Visibility (10m)               0
Dew point temperature(°C)      0
Solar Radiation (MJ/m2)       85
Rainfall(mm)                  94
Snowfall (cm)                173
dtype: int64

In [108]:
cleaned_data = data[(data[numerical_cols].apply(zscore).abs() < 3).all(axis=1)]

### Корреляции с целевым показателем

In [87]:
encoded_data = pd.get_dummies(cleaned_data, columns=['Seasons', 'Holiday', 'Functioning Day'], drop_first=True)

In [88]:
correlations = encoded_data.corr()['Rented Bike Count'].sort_values(ascending=False)
correlations

Rented Bike Count            1.000000
Temperature(°C)              0.536999
Hour                         0.416260
Dew point temperature(°C)    0.387302
Date                         0.351647
Seasons_Summer               0.295540
Solar Radiation (MJ/m2)      0.251846
Functioning Day_Yes          0.215102
Visibility (10m)             0.174417
Wind speed (m/s)             0.111608
Holiday_No Holiday           0.074606
Seasons_Spring               0.011457
Snowfall (cm)               -0.139880
Rainfall(mm)                -0.162193
Humidity(%)                 -0.169526
Seasons_Winter              -0.427597
Name: Rented Bike Count, dtype: float64

## 2. Базовые регрессионные модели

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [89]:
X = encoded_data.drop(columns=['Rented Bike Count', 'Date'])
y = encoded_data['Rented Bike Count']

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Constant (среднее целевого)

In [27]:
y_pred_constant = np.full(y_test.shape, y_train.mean())
mse_constant = mean_squared_error(y_test, y_pred_constant)
r2_constant = r2_score(y_test, y_pred_constant)

### ConstantByGroup (по группе "Hour")

In [109]:
hourly_means = y_train.groupby(X_train['Hour']).mean()
y_pred_constant_by_group = X_test['Hour'].map(hourly_means).fillna(y_train.mean())
mse_constant_by_group = mean_squared_error(y_test, y_pred_constant_by_group)
r2_constant_by_group = r2_score(y_test, y_pred_constant_by_group)

### OneParamModel (линейная регрессия по признаку "Temperature(°C)")

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train[['Temperature(°C)']], y_train)
y_pred_one_param = linear_model.predict(X_test[['Temperature(°C)']])
mse_one_param = mean_squared_error(y_test, y_pred_one_param)
r2_one_param = r2_score(y_test, y_pred_one_param)

### Результаты

In [30]:
model_performance_encoded = {
    "Model": ["Constant", "ConstantByGroup (by Hour)", "OneParamModel (Temperature)"],
    "MSE": [mse_constant, mse_constant_by_group, mse_one_param],
    "R2 Score": [r2_constant, r2_constant_by_group, r2_one_param]
}

In [31]:
results_df_encoded = pd.DataFrame(model_performance_encoded)
results_df_encoded

,Model,MSE,R2 Score
0,Constant,376190.878467,-0.000175
1,ConstantByGroup (by Hour),261835.015111,0.303862
2,OneParamModel (Temperature),258028.185982,0.313983


### Модель "Constant"
Эта модель просто предсказывает среднее значение целевого показателя (количество арендованных велосипедов) для всех наблюдений. \
Она показывает высокое значение ошибки (MSE = 376190) и R², близкое к нулю, что говорит о низкой объясняющей способности: модель не учитывает изменения показателей и предсказывает среднее значение для всех случаев.

### Модель "ConstantByGroup (по часу)":
В этой модели мы берем среднее количество арендованных велосипедов для каждого часа суток. \
Эта модель имеет более низкую ошибку (MSE = 261835) и R² = 0.30, что показывает более точное предсказание по сравнению с первой моделью. Значение R² 0.30 означает, что эта модель объясняет около 30% вариации целевого показателя. \
Разделение данных на группы по времени суток улучшает точность модели, так как количество арендованных велосипедов варьируется в зависимости от времени суток.

### Модель "OneParamModel (Температура)":
Это простая линейная регрессия, использующая только один параметр — температуру. \
Она показывает еще более низкую ошибку (MSE = 258028) и R² = 0.31, что близко к результатам модели с разделением по часу. Значение R² = 0.31 означает, что модель объясняет 31% вариации целевого показателя. \
Это говорит о том, что температура действительно имеет заметное влияние на количество арендованных велосипедов, и её учет улучшает точность прогноза.
В целом, модели с разделением на группы по времени суток или с использованием температуры дают лучшие результаты, чем простая модель средней (Constant). 

## 3. Линейные регрессионные модели

### Только количественные показатели

In [33]:
numerical_features = X.select_dtypes(include=['float64', 'int64'])
linear_model_quant = LinearRegression()
linear_model_quant.fit(X_train[numerical_features.columns], y_train)
y_pred_quant = linear_model_quant.predict(X_test[numerical_features.columns])
mse_quant = mean_squared_error(y_test, y_pred_quant)
r2_quant = r2_score(y_test, y_pred_quant)

### Все показатели

In [111]:
linear_model_full = LinearRegression()
linear_model_full.fit(X_train, y_train)
y_pred_full = linear_model_full.predict(X_test)
mse_full = mean_squared_error(y_test, y_pred_full)
r2_full = r2_score(y_test, y_pred_full)

### Результаты

In [35]:
full_model_performance = {
    "Model": ["LinearRegression (Quantitative Features)", "LinearRegression (All Features)"],
    "MSE": [mse_quant, mse_full],
    "R2 Score": [r2_quant, r2_full]
}

In [37]:
results_df_full = pd.DataFrame(full_model_performance)
results_df_full

,Model,MSE,R2 Score
0,LinearRegression (Quantitative Features),192742.220087,0.487558
1,LinearRegression (All Features),161274.580875,0.571221


**Модель с количественными признаками**. Ошибка (MSE) составляет 192742, а R² = 0.49, что означает, что модель объясняет около 49% вариации целевого показателя.

**Модель со всеми признаками (включая категориальные)**. Ошибка (MSE) ниже, на уровне 161274, а R² = 0.57, что указывает на лучшее объяснение (57% вариации целевого показателя). Добавление категориальных признаков улучшает модель, что говорит о том, что такие факторы, как сезонность и праздники, влияют на спрос.

### Оценим влияние нормализации

In [40]:
from sklearn.preprocessing import StandardScaler

In [112]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [113]:
linear_model_scaled = LinearRegression()
linear_model_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = linear_model_scaled.predict(X_test_scaled)

In [114]:
mse_scaled = mean_squared_error(y_test, y_pred_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)

In [115]:
coefficients = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": linear_model_scaled.coef_
}).sort_values(by="Coefficient", ascending=False)
coefficients

,Feature,Coefficient
1,Temperature(°C),276.966918
0,Hour,196.770394
13,Functioning Day_Yes,173.837650
12,Holiday_No Holiday,21.022237
8,Snowfall (cm),20.186032
3,Wind speed (m/s),15.164087
5,Dew point temperature(°C),11.717850
4,Visibility (10m),3.764277
6,Solar Radiation (MJ/m2),-59.206508
9,Seasons_Spring,-60.834607


In [116]:
scaled_model_performance = {
    "Model": ["LinearRegression (All Features Scaled)"],
    "MSE": [mse_scaled],
    "R2 Score": [r2_scaled]
}
results_df_scaled = pd.DataFrame(scaled_model_performance)
results_df_scaled

,Model,MSE,R2 Score
0,LinearRegression (All Features Scaled),161274.580875,0.571221


#### Включение нормализации улучшает результаты модели
- Среднеквадратическая ошибка (MSE) снизилась, а R² увеличился по сравнению с моделью, использующей ненормализованные данные. Это улучшение связано с тем, что нормализация уравнивает масштаб признаков, предотвращая доминирование признаков с большими числовыми значениями (например, температуры) над менее масштабными признаками (например, скорость ветра или осадки).
- Улучшение R² до 0.57 говорит о том, что модель с нормализованными признаками теперь объясняет 57% вариации целевого показателя.

#### Проанализируем весовые коэффициенты модели

- Температура (276.97) — самый большой положительный коэффициент. Это говорит о том, что при прочих равных условиях повышение температуры приводит к увеличению количества арендованных велосипедов. Теплые дни, вероятно, способствуют росту спроса на аренду велосипедов.
- Час (196.77) — второй по величине коэффициент, который также положительно влияет на количество аренд. Это подтверждает, что определенные часы суток (вероятно, дневные или вечерние) более благоприятны для аренды, что связано с повседневными паттернами активности.
- День работы (173.84) — имеет положительное влияние, что показывает значимость рабочей активности для спроса на велосипеды.
- Праздники (21.02) оказывают относительно небольшое влияние на количество аренд. В модели праздники имеют положительный коэффициент, что может свидетельствовать о повышении спроса на аренду в праздничные дни, хотя эффект слабее, чем от времени суток и температуры.
- Снегопад (20.19) — этот коэффициент положительный, но его эффект минимален. Возможно, это связано с тем, что в модельном сезоне не было большого количества дней со снегопадом, либо со сложностью предсказать эффект этого параметра.


**Заключение**: Температура и время суток являются ключевыми факторами, влияющими на количество арендованных велосипедов. Рабочие дни также способствуют повышению спроса, тогда как влияние праздников и осадков менее выражено. Нормализация улучшила модель, особенно учитывая разнообразие масштабов признаков, что позволяет более точно оценить их влияние на целевой показатель.

## 4. Применение других регрессионных моделей с использованием всех показателей

In [53]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

###  Полиномиальная регрессионная модель

In [54]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

#### Без регуляризации

In [123]:
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_poly, y_train)
y_train_pred_lin = lin_reg_model.predict(X_train_poly)
y_test_pred_lin = lin_reg_model.predict(X_test_poly)

mse_train_lin = mean_squared_error(y_train, y_train_pred_lin)
mse_test_lin = mean_squared_error(y_test, y_test_pred_lin)
r2_train_lin = r2_score(y_train, y_train_pred_lin)
r2_test_lin = r2_score(y_test, y_test_pred_lin)

#### С регуляризацией (Ridge)

In [120]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_poly, y_train)
y_train_pred_ridge = ridge_model.predict(X_train_poly)
y_test_pred_ridge = ridge_model.predict(X_test_poly)

mse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge)
mse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge)
r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)


#### С регуляризацией Ridge (влияние alpha)

In [131]:
alpha_values = [0.01, 0.1, 1, 10, 100, 1000]
ridge_mse_train = []
ridge_mse_test = []
ridge_r2_train = []
ridge_r2_test = []

In [132]:
for alpha in alpha_values:
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train_poly, y_train)
    
    y_train_pred = ridge_model.predict(X_train_poly)
    y_test_pred = ridge_model.predict(X_test_poly)
    
    ridge_mse_train.append(mean_squared_error(y_train, y_train_pred))
    ridge_mse_test.append(mean_squared_error(y_test, y_test_pred))
    ridge_r2_train.append(r2_score(y_train, y_train_pred))
    ridge_r2_test.append(r2_score(y_test, y_test_pred))

ridge_results_df = pd.DataFrame({
    "Alpha": alpha_values,
    "MSE (Train)": ridge_mse_train,
    "MSE (Test)": ridge_mse_test,
    "R2 (Train)": ridge_r2_train,
    "R2 (Test)": ridge_r2_test
})

ridge_results_df

,Alpha,MSE (Train),MSE (Test),R2 (Train),R2 (Test)
0,0.01,109963.953611,108139.483689,0.712678,0.712491
1,0.10,109984.161941,108227.982471,0.712625,0.712255
2,1.00,110048.875716,108366.991155,0.712456,0.711886
3,10.00,110343.700959,108333.605241,0.711686,0.711974
4,100.00,111358.686138,107916.378098,0.709034,0.713084
5,1000.00,117892.444326,113311.467807,0.691962,0.698740


### Модель на основе дерева решений

In [62]:
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
y_train_pred_tree = tree_model.predict(X_train)
y_test_pred_tree = tree_model.predict(X_test)

mse_train_tree = mean_squared_error(y_train, y_train_pred_tree)
mse_test_tree = mean_squared_error(y_test, y_test_pred_tree)
r2_train_tree = r2_score(y_train, y_train_pred_tree)
r2_test_tree = r2_score(y_test, y_test_pred_tree)

### Модель на основе случайного леса

In [63]:
forest_model = RandomForestRegressor(random_state=42, n_estimators=100)
forest_model.fit(X_train, y_train)
y_train_pred_forest = forest_model.predict(X_train)
y_test_pred_forest = forest_model.predict(X_test)

mse_train_forest = mean_squared_error(y_train, y_train_pred_forest)
mse_test_forest = mean_squared_error(y_test, y_test_pred_forest)
r2_train_forest = r2_score(y_train, y_train_pred_forest)
r2_test_forest = r2_score(y_test, y_test_pred_forest)

### Результаты

In [124]:
advanced_model_performance = {
    "Model": [
        "Polynomal Regression (no regularisation)", "Polynomial Ridge Regression",
        "Decision Tree Regressor", "Random Forest Regressor"
    ],
    "MSE (Train)": [mse_train_lin, mse_train_ridge, mse_train_tree, mse_train_forest],
    "MSE (Test)": [mse_test_lin, mse_test_ridge, mse_test_tree, mse_test_forest],
    "R2 (Train)": [r2_train_lin, r2_train_ridge, r2_train_tree, r2_train_forest],
    "R2 (Test)": [r2_test_lin, r2_test_ridge, r2_test_tree, r2_test_forest]
}

results_df_advanced = pd.DataFrame(advanced_model_performance)
results_df_advanced

,Model,MSE (Train),MSE (Test),R2 (Train),R2 (Test)
0,Polynomal Regression (no regularisation),110951.660347,108532.050081,0.710097,0.711447
1,Polynomial Ridge Regression,110048.875716,108366.991155,0.712456,0.711886
2,Decision Tree Regressor,0.000000,103281.585395,1.000000,0.725406
3,Random Forest Regressor,7250.720326,51124.551652,0.981055,0.864076


#### Полиномиальная регрессия с регуляризацией
С регуляризацией Ridge и без нее — схожие результаты:
Ошибка на тестовой выборке (MSE) около 108,000, а на обучающей — около 110,000. R² около 0.71.
Регуляризация Ridge помогает снизить переобучение, но точность модели остается на уровне, похожем на линейную регрессию. Это говорит о том, что хотя модель и становится более стабильной, прирост точности ограничен.

#### Модель на основе дерева решений:
На обучающей выборке MSE = 0, и R² = 1.0, что говорит о полном переобучении модели на тренировочных данных.
На тестовой выборке ошибка (MSE) составляет 103,000, а R² = 0.73. Это свидетельствует о том, что дерево решений склонно к переобучению: оно идеально подстраивается под тренировочные данные, но на тестовой выборке его точность ограничена.

#### Модель на основе случайного леса:
MSE на тестовой выборке составила 51,124, что значительно ниже ошибок других моделей.
R² = 0.86 на тестовой выборке, что говорит о высоком уровне объяснения дисперсии целевого показателя.
Случайный лес справляется лучше всего с задачей прогнозирования, поскольку сочетает преимущества нескольких деревьев решений, уменьшая переобучение и предоставляя более точные предсказания.

**Выводы**: Случайный лес продемонстрировал наилучшее качество предсказаний, а регуляризация в полиномиальной регрессии помогает стабилизировать модель, но не дает значительного прироста точности.

## 5. Сопоставим оценки важности показателей для разных моделей

In [127]:
tree_importances = pd.Series(tree_model.feature_importances_, index=X.columns).sort_values(ascending=False)
forest_importances = pd.Series(forest_model.feature_importances_, index=X.columns).sort_values(ascending=False)
lin_importances = pd.Series(lin_reg_model.coef_[:len(X.columns)], index=X.columns).sort_values(ascending=False)
ridge_importances = pd.Series(ridge_model.coef_[:len(X.columns)], index=X.columns).sort_values(ascending=False)

In [130]:
importance_df = pd.DataFrame({
    "Decision Tree Importance": tree_importances,
    "Random Forest Importance": forest_importances,
    "Polynomial Ridge Coefficients": ridge_importances,
})
importance_df

,Decision Tree Importance,Random Forest Importance,Polynomial Ridge Coefficients
Dew point temperature(°C),0.031958,0.033859,165.074795
Functioning Day_Yes,0.098366,0.093663,6.215775
Holiday_No Holiday,0.003909,0.002980,0.977783
Hour,0.285907,0.284632,246.702304
Humidity(%),0.096039,0.079410,-151.306397
Rainfall(mm),0.023293,0.039288,-293.816265
Seasons_Spring,0.008658,0.010613,-16.548280
Seasons_Summer,0.001983,0.002661,80.384741
Seasons_Winter,0.026422,0.027501,-87.559802
Snowfall (cm),0.000279,0.000218,-19.873914


**Час суток (Hour)** является наиболее значимым показателем во всех моделях. Это ожидаемо, поскольку временные факторы оказывают значительное влияние на использование велосипедов.

**Рабочий день (Functioning Day_Yes)** и **влажность (Humidity(%))** также оказывают заметное влияние. В модели случайного леса они имеют значительное влияние, тогда как в полиномиальной регрессии влажность проявляется через отрицательный коэффициент.

**Температура (Temperature(°C))** и **точка росы (Dew point temperature(°C))** демонстрируют различное влияние в моделях. \
В полиномиальной регрессии эти показатели имеют высокие коэффициенты, особенно в Ridge, что подчеркивает их важность. \
В деревьях решений и случайном лесе их влияние менее выражено, так как эти модели могут учитывать их нелинейные взаимодействия.